In [1]:
import numpy as np
import scipy.io as scio
from PIL import Image
import math
import matplotlib.pyplot as plt

def Find_K_Max_Eigen(Matrix,Eigen_NUM):
    #Matrix方阵
    row=Matrix.shape[0]
    col=Matrix.shape[1]
    #V：Matrix的特征向量   S:Matrix的特征值
    S,V=np.linalg.eig(Matrix)
    S=S.reshape((-1,1),order='F')
    
    sorted_S=sorted(enumerate(S), key=lambda x: x[1])
    index = [i[0] for i in sorted_S]
    S = [i[1] for i in sorted_S]
    #初始化特征向量与特征值
    Eigen_Vector=np.zeros((row,Eigen_NUM))
    Eigen_Value=np.zeros(Eigen_NUM)
    p=row-1; #p表示Matrix的大小
    for t in range(Eigen_NUM):
        Eigen_Vector[:,t]=V[:,index[p]]
        Eigen_Value[t]=S[p]
        p=p-1
    return Eigen_Vector,Eigen_Value


def Eigenface_f(Train_SET,Eigen_NUM):
    row=Train_SET.shape[0]
    col=Train_SET.shape[1]
    
    Mean_Image=np.mean(Train_SET,axis=1)#取每一行的均值   size=NN*1
    Mean_Image=Mean_Image.reshape((-1,1),order='F')
    Train_SET=Train_SET-np.dot(Mean_Image,np.ones((1,col)))#Train_SET中每一行减去对应行的均值
    #行数小于列数
    if row<=col:
        R=np.dot(Train_SET,Train_SET.T)/(col-1) #size(R)=NN*NN
        V,S=Find_K_Max_Eigen(R,Eigen_NUM)
        disc_value=S#R中前Eigen_NUM个最大的特征值
        disc_set=V#R中前Eigen_NUM个最大的特征向量
    else:
        R=np.dot(Train_SET.T,Train_SET)/(col-1)#size(R)=Train_NUM*Train_NUM 
        V,S=Find_K_Max_Eigen(R,Eigen_NUM)

        disc_value=S;#R中前Eigen_NUM个最大的特征值
        disc_set=np.zeros((row,Eigen_NUM))
        Train_SET=Train_SET/math.sqrt(col-1)
        for k in range(Eigen_NUM):
            disc_set[:,k]=(1/math.sqrt(disc_value[k]))*np.dot(Train_SET,V[:,k].reshape((-1,1),order='F')).T
    return disc_set,disc_value,Mean_Image

In [2]:
#读取训练文件
train_data=scio.loadmat('database/AR_database.mat')
train_size=train_data['NewTrain_DAT'].shape   # 2580*700
print(train_size)
#构造训练集 (无表情人脸)
idx=[]
train_label=[]
for i in range(100):
    idx.extend([i*7,i*7+4,i*7+5,i*7+6])
    train_label.extend([i,i,i,i])
train_set=train_data['NewTrain_DAT'][:,idx]
train_label=np.array(train_label)
# train_label=train_data['trainlabels'][0,idx]

#读取测试文件
test_data=scio.loadmat('database/AR_database_Occlusion.mat')
test_size=test_data['Tt_dataMatrix'].shape
#构造测试集
idx1=[]
test_label=[]
for i in range(100):
    idx1.extend([i*6+3,i*6+4,i*6+5])
    test_label.extend([i,i,i])
test_set=test_data['Tt_dataMatrix'][:,idx1]
test_label=np.array(test_label)
# test_label=test_data['Tt_sampleLabels'][]


#训练集                      
train_img_h=60
train_img_w=43
p_train_set=[]
for i in range(train_set.shape[1]):
    image_m=(train_set[:,i].reshape((train_img_h,train_img_w),order='F'))
    im=Image.fromarray(image_m)
    image_m=np.array(im.resize((30,42),Image.NEAREST))
#     im=Image.fromarray(image_m)
#     im=im.convert('L')
#     im.show()
    image_m=image_m.reshape((-1,1),order='F')
    image_m=image_m.astype('uint8')
    p_train_set.append(image_m)      #训练集列表

p_train_set=np.array(p_train_set)    #便于后续的矩阵相乘
p_train_set=p_train_set[:,:,0].T
print(p_train_set.shape)

#测试集
test_img_h=165
test_img_w=120
p_test_set=[]
for i in range(test_set.shape[1]):
    image_m=(test_set[:,i].reshape((test_img_h,test_img_w),order='F'))
    im=Image.fromarray(image_m)
    image_m=np.array(im.resize((30,42),Image.NEAREST))
    image_m=image_m.reshape((-1,1),order='F')
    image_m=image_m.astype('uint8')
    p_test_set.append(image_m)

p_test_set=np.array(p_test_set)
# print(p_test_set.shape)
p_test_set=p_test_set[:,:,0].T
# print(p_test_set.shape)
                      
eig_set,eig_value,mean_Image=Eigenface_f(p_train_set,260)
eig_value=eig_value.reshape((-1,1),order='F')
# print(eig_value.shape)
# print(eig_set.shape)
eig_value=np.sqrt(eig_value)
# print(mean_Image.shape)
x_mean=(mean_Image+0.001*np.dot(eig_set,eig_value))
# print(x_mean)
x_img_array=x_mean.reshape((42,30),order='F')
# print(x_img_array)
x_img=Image.fromarray(x_img_array)
x_img.show()

(2580, 700)
(1260, 400)


In [ ]:
train_label_set=set(list(train_label))
#class_list=np.array(list(range(1,len(train_label)+1)))
lamda=0.0001
ecsep=8
tao=0.6

p_train_set=p_train_set/(np.sqrt(np.sum(p_train_set*p_train_set)))

# 算法流程
identify=[]
for i in range(p_test_set.shape[1]):
    print(i)
    #第一步
    y=p_test_set[:,i].reshape(-1,1,order='F')
#     print(y.shape)
#     x_img_array=y.reshape(42,30)
#     x_img=Image.fromarray(x_img_array)
#     x_img.show()
    #====================RRC_L1 begin=========================
    e=(y-x_mean)**2
    e_sorted=np.sort(e,axis=0)
    n=p_test_set.shape[0]
    thresh=e_sorted[math.ceil(tao*n)-1]
    miu=ecsep/thresh
    #第二步
    w=1/(1+1/np.exp(-1*miu*(e-thresh)))
    w_pre=w
    
    norm_y=np.linalg.norm(y)
    y=y/ norm_y
    y=y.reshape(-1,1,order='F')

    #第三步
    nIter=15
    for nit in range(nIter):
        tem_w=w/max(w)

        idx=[j for j in range(len(tem_w)) if tem_w[j]>1e-3]
        w_p=w[idx]
        y_p=y[idx]
        train_set_p=p_train_set[idx]

        alafa_p=np.zeros([p_train_set.shape[1],1])
        v=np.ones([p_train_set.shape[1],1])
        alafa=np.ones([p_train_set.shape[1],1])
        E=np.eye(p_train_set.shape[1])
        ecs_pre=1000

        w_p=w_p.reshape((-1,1),order='F')
        y_p=y_p.reshape((-1,1),order='F')
        # print(v.shape,w_p.shape,train_set_p.shape,E.shape,y_p.shape)
        innerit=0
        while np.linalg.norm(alafa-alafa_p,2)/np.linalg.norm(alafa,2) >1e-2 and innerit <=50:
            alafa_p=alafa
            innerit+=1
            alafa=v*np.dot(np.linalg.inv(np.dot((w_p*train_set_p).T,(w_p*train_set_p))*v.T+lamda*np.linalg.norm(w_p*y_p)*E),np.dot((w_p*train_set_p).T,(w_p*y_p)))
            sorted_alafa=np.sort(abs(alafa),axis=0)
            ecs=abs(sorted_alafa[math.ceil(0.01*p_train_set.shape[1])-1])
            ecs=min(ecs_pre,ecs/p_train_set.shape[1])
            ecs_pre=ecs
            v=np.sqrt(alafa**2+ecs**2)

        
        # print(y.shape,p_train_set.shape,alafa.shape)
        e=(np.linalg.norm(y)**2)*((y-np.dot(p_train_set,alafa)))
        e_sorted=np.sort(e,axis=0)
        thresh=e_sorted[math.ceil(tao*len(e))-1]
        miu=ecsep/thresh
        # print(e.shape)
        w=1/(1+1/np.exp(-1*miu*(e-thresh)))
        # print(w.shape)
        
    cur_min=-1
    min_idx=-1
    for k in range(len(train_label_set)):
        idx=(train_label==k)
        z=w*(y-np.dot(p_train_set[:,idx],alafa[idx]))
        cur=np.dot(z.T,z)
        if cur_min==-1:
            cur_min=cur
            min_idx=k
        elif cur<cur_min:
            cur_min=cur
            min_idx=k
    identify.append(min_idx)
    
reco_rate=sum(identify==test_label)/len(test_label)

In [ ]:
len(identify)

In [ ]:
reco_rate=0
for i in range(len(identify)):
    if identify[i]==test_label[i]:
        reco_rate+=1
reco_rate/len(identify)

In [ ]:
train_data['NewTrain_DAT'][:,0]